# Sonecule CPMSonCB: Continuous PMSon - score-based with callback function for flexible mapping

Using a continous parameter mapping sonification with a callback function to specify the mapping



## Data Preparation

In [ ]:
import sonecules as sn
sn.startup()
sn.pb = sn.playback
ctx = sn.gcc()
import sc3nb as scn
from pya import Asig
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

import sc3nb as scn
def a2dict(**kwargs):
    """turn argument list into dictionary"""
    return kwargs

In [ ]:
%run ../examples/prepare-data.ipynb

In [ ]:
df = bld_df
df.plot(lw=0.2);

## Sonecule pre-development: synth and control

In [ ]:
# create synth for tvosc
scn.SynthDef("contsyn", 
"""{ | out=0, freq=400, amp=0.1, vibfreq=0, vibintrel=0, numharm=0, pulserate=0, pint=0, pwid=1, pan=0 | 
    var vib = SinOsc.ar(vibfreq, mul: vibintrel*freq, add: freq);
    var sig = Blip.ar(vib, mul: amp, numharm: numharm);
    var pulse = LFPulse.ar(freq: pulserate, iphase: 0.0, width: pwid, mul: pint, add: 1-pint);
    Out.ar(out, Pan2.ar(sig * pulse, pan));
}""").add();

In [ ]:
ctx.enable_realtime();

In [ ]:
syn = ctx.synths.create(name="contsyn", track=1,  mutable=True)
syn

In [ ]:
syn.start(freq=400, amp=0.1, vibfreq=3, vibintrel=0.1, numharm=5, pulserate=15, pint=0.5, pwid=0.5, pan=0)


In [ ]:
# change via properties
syn.freq = 323
syn.numharm = 1
syn.pint = 0
syn.vibintrel = 0.01
syn.vibfreq = 6


In [ ]:
# set via dictionary
syn.set({'freq': 600, 'numharm': 2, 'pint': 0.2, 'amp': 0.05})

In [ ]:
syn.stop()

In [ ]:
ctx.backend.stop(ctx)

In [ ]:
ctx.disable_realtime()

In [ ]:
# set mesonic backend latency
sn.pb().processor.latency = 0.1

## Sonecule Pre-development - loop and player

In [ ]:
df = bld_df.iloc[:24*7]

In [ ]:
# create synths
syn = ctx.synths.create(name="contsyn", track=1,  mutable=True)

In [ ]:
df.columns

In [ ]:
syn.params.keys()

In [ ]:
# here schedule function, with argument for replace default to true
ctx.reset()

# start syns (oscillators) with default values
pdict = {}
for k, v in syn.params.items():
    pdict[k] = v.default

with ctx.at(time=0):
    syn.start(**pdict)

# service mapcol function
def mapcol(r, name, cmins, cmaxs, dmi, dma):
    return scn.linlin(r[name], cmins[name], cmaxs[name], dmi, dma)

# default callback function (later submitted)
def callback_fn(r, cmi, cma, pp):
    pp['freq']      = scn.midicps(mapcol(r, 'temperature', cmi, cma, 48, 72))
    pp['amp']       = mapcol(r, 'humidity', cmi, cma, 0.1, 0.9) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['numharm']   = mapcol(r, 'solar_radiation', cmi, cma, 1, 6)
    pp['vibfreq']   = scn.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibintrel'] = 0.05
    return pp

# schedule function
duration = 14
maxonset = -1
nrows = df.shape[0]
cmi = df.min()
cma = df.max()
# modulate parameters by data 
for idx, r in df.iterrows():
    onset = scn.linlin(idx, 0, nrows, 0, duration)
    with ctx.at(time=onset):
        pardict = callback_fn(r, cmi, cma, pdict)
        syn.set(pardict)
    if onset > maxonset:
        maxonset = onset

# stop oscillator at end
with ctx.at(time=maxonset):
    syn.stop()

# start sonification playback
sn.pb().start()

## Implementation

In [ ]:
from mesonic.synth import Synth
from sonecules.base import Sonecule
import numpy, numbers

In [ ]:
class CPMSonCB(Sonecule):
    def __init__(self, data, synthdef=None, context=None):
        super().__init__(context=context)

        self.data = data
        self.synthdef = synthdef
        if self.synthdef:
            scn.SynthDef("contsyn", synthdef).add()
        else:
            print("no synth definition: use default contsyn")
            scn.SynthDef("contsyn", 
            """{ | out=0, freq=400, amp=0.1, vibfreq=0, vibintrel=0, numharm=0, pulserate=0, pint=0, pwid=1, pan=0 | 
                var vib = SinOsc.ar(vibfreq, mul: vibintrel*freq, add: freq);
                var sig = Blip.ar(vib, mul: amp, numharm: numharm);
                var pulse = LFPulse.ar(freq: pulserate, iphase: 0.0, width: pwid, mul: pint, add: 1-pint);
                Out.ar(out, Pan2.ar(sig * pulse, pan));
            }""").add()

        ctx = self.context

        ctx._backend.sc.server.sync()

        self.syn = ctx.synths.create(name="contsyn", track=1,  mutable=True)

        self.pdict = {}
        for k, v in self.syn.params.items():
            self.pdict[k] = v.default


    @staticmethod
    def mapcol(r, name, cmins, cmaxs, dmi, dma):
        """service mapcol function"""
        return scn.linlin(r[name], cmins[name], cmaxs[name], dmi, dma)

    def schedule(
        self,
        at=0,
        duration=4,
        callback_fn=None,
        reset_flag=True,
    ):
        # here schedule function, with argument for replace default to true
        # "change"
        ctx = self.context
        if reset_flag:
            sn.reset() 

        # create synths
        with ctx.at(time=at):
            self.syn.start(**self.pdict)

        # compute parameters for mapping
        # 1. src ranges for pitch mapping

        df = self.data
        maxonset = -1
        nrows = df.shape[0]
        cmi = df.min()
        cma = df.max()
        # modulate parameters by data 
        ct = 0 
        for idx, r in df.iterrows():
            onset = scn.linlin(ct, 0, nrows, 0, duration)
            with ctx.at(time=at+onset):
                pdict = callback_fn(r, cmi, cma, self.pdict)
                self.syn.set(pdict)
            if onset > maxonset:
                maxonset = onset
            ct += 1

        # stop oscillators

        # stop oscillator at end
        with ctx.at(time=at+maxonset):
            self.syn.stop()

        return self
    
    def create_callback_template(self, auto_assign=False):
        df = self.data
        tabstr = "    "
        str = "def cbfn(r, cmi, cma, pp):\n"
        str += tabstr + f"# columns are:" 
        feature_list = []
        for i, col in enumerate(df.columns):
            feature = col
            feature_list.append(feature)
            str += f"'{col}' "
            if (i+1) % 4 == 0: 
                str += "\n" + tabstr + '# '
        str += '\n'
        
        fct = 0
        for p in self.pdict:
            if p == 'out': 
                continue
            if auto_assign:
                # assign features automatically
                feature = feature_list[fct]
                fct += 1
                if fct == len(feature_list)-1:
                    fct = 0
                leftstr = f"pp['{p}']"
                bound_left = self.pdict[p]*0.75
                bound_right = self.pdict[p]*1.5
                str += tabstr + f"{leftstr:15s}\t = mapcol(r, '{feature}', cmi, cma, {bound_left}, {bound_right})\n"
                pass
            else:
                str += tabstr + f"pp['{p}']\t = mapcol(r, 'colname', cmi, cma, 1, 2)\n"
            ""
        print(str)
        print("# create sonification e.g. by using\n" +
              "scb.schedule(at=0, duration=2, callback_fn=callback_fn).start(rate=1)\n")
        return str

    def start(self, **kwargs):
        """start sonification rendering by starting the playback
        kwargs are passed on to start(), so use rate to control speedup, etc.
        """
        print(kwargs)
        sn.playback().start(**kwargs)
    

## Usage / Demo Snippets

First let's craft a continuous PMSon using a callback function for mapping

In [ ]:
# scb = CPMSonCB(bld_df.iloc[:7*24, 6:])  # one week
scb = CPMSonCB(bld_df.iloc[:4000, 7:])  # whole dataset

mapcol = scb.mapcol

def callback_fn(r, cmi, cma, pp):
    pp['freq']      = scn.midicps(mapcol(r, 'temperature', cmi, cma, 48, 72))
    pp['amp']       = scn.dbamp(mapcol(r, 'humidity', cmi, cma, -20, 0)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['numharm']   = mapcol(r, 'solar_radiation', cmi, cma, 1, 12)
    pp['vibfreq']   = scn.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibintrel'] = 0
    return pp

scb.schedule(at=0, duration=4, callback_fn=callback_fn).start(rate=1)

@Dennis: it sounds like mesonic has timing issues: if I play 4000 rows in 4 seconds, the sonification needs more than 4 seconds
- any ideas?

Now let the sonecule propose a mapping as starting point for own experiments.

- Calling `create_callback_template()` composes 
- and then prints the python code string.
- This output can be copied into a notebook cell and adapted as needed.
- On execution it defines the callback function cbfn().
- The sonification can be rendered as you go until satisfactory.

In [ ]:
scb = CPMSonCB(bld_df.iloc[:7*24, 8:])  # whole dataset but no weekday features

fnstr = scb.create_callback_template(auto_assign=True)

In [ ]:
def cbfn(r, cmi, cma, pp):
    # columns are:'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    pp['freq']     	 = mapcol(r, 'hour', cmi, cma, 300.0, 600.0)
    pp['amp']      	 = mapcol(r, 'am_pm', cmi, cma, 0.07500000111758709, 0.15000000223517418)
    pp['vibfreq']  	 = mapcol(r, 'temperature', cmi, cma, 0.0, 0.0)
    pp['vibintrel']	 = mapcol(r, 'humidity', cmi, cma, 0.0, 0.0)
    pp['numharm']  	 = mapcol(r, 'solar_radiation', cmi, cma, 0.0, 0.0)
    pp['pulserate']	 = mapcol(r, 'wind_speed', cmi, cma, 0.0, 0.0)
    pp['pint']     	 = mapcol(r, 'hc_wb_electrical', cmi, cma, 0.0, 0.0)
    pp['pwid']     	 = mapcol(r, 'hc_wb_cold_water', cmi, cma, 0.75, 1.5)
    pp['pan']      	 = mapcol(r, 'hour', cmi, cma, 0.0, 0.0)

# create sonification e.g. by using
scb.schedule(at=0, duration=2, callback_fn=callback_fn).start(rate=1)


And finally here is a hand-crafted mapping, from modifying the code

In [ ]:
scb = CPMSonCB(bld_df.iloc[14*24:18*24, 8:]) 

def cbfn(r, cmi, cma, pp):
    # columns are:
    # 'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    # print(r, pp)
    pp['freq']	     = mapcol(r, 'solar_radiation', cmi, cma, 100, 800)
    pp['amp']	     = mapcol(r, 'humidity', cmi, cma, 0, 1)
    pp['numharm']	 = mapcol(r, 'am_pm', cmi, cma, 1, 4)
    pp['vibintrel']  = 0
    pp['pan']	     = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    return pp

scb.schedule(at=1, duration=8, callback_fn=cbfn).start(rate=1)

In [ ]:
ctx.backend.stop(ctx)